<a href="https://colab.research.google.com/github/orekhovsky/GenAI-mini-projects/blob/main/text2sql_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install rouge-score -q

  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
from google.colab import userdata
df = pd.read_json("hf://datasets/Clinton/Text-to-sql-v1/texttosqlv2.jsonl", lines=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df

,instruction,input,response,source,text
0,Name the home team for carlton away team,CREATE TABLE table_name_77 (\n home_team VA...,SELECT home_team FROM table_name_77 WHERE away...,sql_create_context,Below are sql tables schemas paired with instr...
1,what will the population of Asia be when Latin...,"CREATE TABLE table_22767 (\n ""Year"" real,\n...","SELECT ""Asia"" FROM table_22767 WHERE ""Latin Am...",wikisql,Below are sql tables schemas paired with instr...
2,How many faculty members do we have for each g...,"CREATE TABLE Student (\n StuID INTEGER,\n ...","SELECT Sex, COUNT(*) FROM Faculty GROUP BY Sex...",nvbench,Below are sql tables schemas paired with instr...
3,List the record of 0-1 from the table?,CREATE TABLE table_14656147_2 (\n week VARC...,SELECT week FROM table_14656147_2 WHERE record...,sql_create_context,Below are sql tables schemas paired with instr...
4,"Which silver has a Gold smaller than 12, a Ran...",CREATE TABLE table_name_24 (\n silver VARCH...,SELECT silver FROM table_name_24 WHERE gold < ...,sql_create_context,Below are sql tables schemas paired with instr...
...,...,...,...,...,...
262203,Most viewed posts by tag w/o tag in post title.,"CREATE TABLE Comments (\n Id number,\n P...","SELECT Id AS ""post_link"", Tags, ViewCount, Cre...",sede,Below are sql tables schemas paired with instr...
262204,What is the time when ss12 is stage?,"CREATE TABLE table_24485 (\n ""Day"" text,\n ...","SELECT ""Time"" FROM table_24485 WHERE ""Stage"" =...",wikisql,Below are sql tables schemas paired with instr...
262205,count the number of patients whose year of dea...,CREATE TABLE demographic (\n subject_id tex...,SELECT COUNT(DISTINCT demographic.subject_id) ...,mimicsql_data,Below are sql tables schemas paired with instr...
262206,What was the styled used to defeat the opponen...,CREATE TABLE table_18598175_2 (\n method_of...,SELECT method_of_elimination FROM table_185981...,sql_create_context,Below are sql tables schemas paired with instr...


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
import pandas as pd
import requests
import json
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer

# Функция для вызова модели через API
def generate_sql_code(instruction, table_schema, api_key):
    prompt = f"""
    You are an expert SQL query generator. Answer only with valid SQL code, nothing else.
    Instruction: {instruction}
    Table Schema: {table_schema}
    SQL code:
    """
    input_data = {
        "model": "meta-llama/llama-3.2-1b-instruct",
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps(input_data)
    )
    if response.status_code == 200:
        data = response.json()
        return data.get('choices', [{}])[0].get('message', {}).get('content', 'No content found')
    else:
        raise RuntimeError(f"API request failed with status {response.status_code}: {response.text}")

# Функция для вычисления метрик
def calculate_metrics(true_answer, model_response):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(true_answer, model_response)

    true_tokens = true_answer.split()
    response_tokens = model_response.split()
    smooth_fn = SmoothingFunction().method1
    bleu_score = sentence_bleu([true_tokens], response_tokens, smoothing_function=smooth_fn)

    meteor = meteor_score([true_answer.split()], model_response.split())

    return bleu_score, meteor, rouge_scores['rouge1'].fmeasure, rouge_scores['rouge2'].fmeasure, rouge_scores['rougeL'].fmeasure

import sqlite3

def test_sql_query(table_schema, sql_query):
    try:
        # Создаем временную базу данных SQLite в памяти
        conn = sqlite3.connect(":memory:")
        cursor = conn.cursor()

        # Выполняем создание таблицы из схемы
        cursor.executescript(table_schema)

        # Выполняем сгенерированный SQL-запрос
        cursor.execute(sql_query)
        result = cursor.fetchall()

        # Закрываем соединение
        conn.close()

        return {"success": True, "result": result}
    except Exception as e:
        return {"success": False, "error": str(e)}

# Основной код
def evaluate_model(df, api_key, model_name="SQL Query Generator", summary_df=None):
    metrics_data = []  # Список для хранения строк данных
    for i, row in df.head(40).iterrows():  # Обрабатываем первые 40 строк
        instruction = row['instruction']
        table_schema = row['input']  # Используем описание таблицы
        true_response = row['response']

        try:
            # Генерация ответа модели
            model_response = generate_sql_code(instruction, table_schema, api_key)

            # Тестируем SQL-код
            test_result = test_sql_query(table_schema, model_response)
            sql_valid = test_result["success"]
            sql_error = test_result.get("error", "")

            # Вычисляем метрики
            bleu, meteor, rouge1, rouge2, rougeL = calculate_metrics(true_response, model_response)

            # Добавляем строку данных в список
            metrics_data.append({
                "Instruction": instruction,
                "Response": true_response,
                "Model Response": model_response,
                "BLEU": bleu,
                "METEOR": meteor,
                "ROUGE-1": rouge1,
                "ROUGE-2": rouge2,
                "ROUGE-L": rougeL,
                "SQL Valid": sql_valid,
                "SQL Error": sql_error
            })
        except Exception as e:
            print(f"Error processing row {i}: {e}")

    # Создаем датафрейм с метриками для каждого примера
    metrics_df = pd.DataFrame(metrics_data)

    # Рассчитываем средние значения метрик и процент правильных ответов
    average_bleu = metrics_df['BLEU'].mean()
    average_meteor = metrics_df['METEOR'].mean()
    average_rouge1 = metrics_df['ROUGE-1'].mean()
    average_rouge2 = metrics_df['ROUGE-2'].mean()
    average_rougeL = metrics_df['ROUGE-L'].mean()
    correct_responses_percentage = (metrics_df['SQL Valid'].sum() / len(metrics_df)) * 100

    # Создаем строку с метриками для текущей модели
    summary_data = {
        "Model": model_name,
        "Average BLEU": average_bleu,
        "Average METEOR": average_meteor,
        "Average ROUGE-1": average_rouge1,
        "Average ROUGE-2": average_rouge2,
        "Average ROUGE-L": average_rougeL,
        "Correct Responses (%)": correct_responses_percentage
    }

    # Если передан существующий summary_df, добавляем строку
    if summary_df is not None:
        summary_df = pd.concat([summary_df, pd.DataFrame([summary_data])], ignore_index=True)
    else:
        # Создаем новый датафрейм, если его нет
        summary_df = pd.DataFrame([summary_data])

    return metrics_df, summary_df

# Пример использования
OPENROUTER_API_KEY = userdata.get('OPEN_Router')

# Допустим, ранее был создан summary_df
previous_summary_df = pd.DataFrame()

# Оцениваем новую модель и обновляем summary_df
metrics_df, summary_df = evaluate_model(df, OPENROUTER_API_KEY, model_name="llama-3.2-1b-instruct", summary_df=previous_summary_df)

# Результаты
summary_df



,Model,Average BLEU,Average METEOR,Average ROUGE-1,Average ROUGE-2,Average ROUGE-L,Correct Responses (%)
0,llama-3.2-1b-instruct,0.347761,0.647689,0.780819,0.640046,0.751439,0.0


In [ ]:
import pandas as pd
import requests
import json
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer

# Функция для вызова модели через API
def generate_sql_code(instruction, table_schema, api_key, model):
    prompt = f"""
    You are an expert SQL query generator. Answer only with valid SQL code, nothing else.
    Instruction: {instruction}
    Table Schema: {table_schema}
    SQL code:
    """
    input_data = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps(input_data)
    )
    if response.status_code == 200:
        data = response.json()
        return data.get('choices', [{}])[0].get('message', {}).get('content', 'No content found')
    else:
        raise RuntimeError(f"API request failed with status {response.status_code}: {response.text}")

# Функция для вычисления метрик
def calculate_metrics(true_answer, model_response):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(true_answer, model_response)

    true_tokens = true_answer.split()
    response_tokens = model_response.split()
    smooth_fn = SmoothingFunction().method1
    bleu_score = sentence_bleu([true_tokens], response_tokens, smoothing_function=smooth_fn)

    meteor = meteor_score([true_answer.split()], model_response.split())

    return bleu_score, meteor, rouge_scores['rouge1'].fmeasure, rouge_scores['rouge2'].fmeasure, rouge_scores['rougeL'].fmeasure

import sqlite3

def test_sql_query(table_schema, sql_query):
    try:
        # Создаем временную базу данных SQLite в памяти
        conn = sqlite3.connect(":memory:")
        cursor = conn.cursor()

        # Выполняем создание таблицы из схемы
        cursor.executescript(table_schema)

        # Выполняем сгенерированный SQL-запрос
        cursor.execute(sql_query)
        result = cursor.fetchall()

        # Закрываем соединение
        conn.close()

        return {"success": True, "result": result}
    except Exception as e:
        return {"success": False, "error": str(e)}

# Основной код
def evaluate_model(df, api_key, model="meta-llama/llama-3.2-1b-instruct", summary_df=None):
    metrics_data = []  # Список для хранения строк данных
    for i, row in df.head(40).iterrows():  # Обрабатываем первые 40 строк
        instruction = row['instruction']
        table_schema = row['input']  # Используем описание таблицы
        true_response = row['response']

        try:
            # Генерация ответа модели
            model_response = generate_sql_code(instruction, table_schema, api_key, model)

            # Тестируем SQL-код
            test_result = test_sql_query(table_schema, model_response)
            sql_valid = test_result["success"]
            sql_error = test_result.get("error", "")

            # Вычисляем метрики
            bleu, meteor, rouge1, rouge2, rougeL = calculate_metrics(true_response, model_response)

            # Добавляем строку данных в список
            metrics_data.append({
                "Instruction": instruction,
                "Response": true_response,
                "Model Response": model_response,
                "BLEU": bleu,
                "METEOR": meteor,
                "ROUGE-1": rouge1,
                "ROUGE-2": rouge2,
                "ROUGE-L": rougeL,
                "SQL Valid": sql_valid,
                "SQL Error": sql_error
            })
        except Exception as e:
            print(f"Error processing row {i}: {e}")

    # Создаем датафрейм с метриками для каждого примера
    metrics_df = pd.DataFrame(metrics_data)

    # Рассчитываем средние значения метрик и процент правильных ответов
    average_bleu = metrics_df['BLEU'].mean()
    average_meteor = metrics_df['METEOR'].mean()
    average_rouge1 = metrics_df['ROUGE-1'].mean()
    average_rouge2 = metrics_df['ROUGE-2'].mean()
    average_rougeL = metrics_df['ROUGE-L'].mean()
    correct_responses_percentage = (metrics_df['SQL Valid'].sum() / len(metrics_df)) * 100

    # Создаем строку с метриками для текущей модели
    summary_data = {
        "Model": model,
        "Average BLEU": average_bleu,
        "Average METEOR": average_meteor,
        "Average ROUGE-1": average_rouge1,
        "Average ROUGE-2": average_rouge2,
        "Average ROUGE-L": average_rougeL,
        "Correct Responses (%)": correct_responses_percentage
    }

    # Если передан существующий summary_df, добавляем строку
    if summary_df is not None:
        summary_df = pd.concat([summary_df, pd.DataFrame([summary_data])], ignore_index=True)
    else:
        # Создаем новый датафрейм, если его нет
        summary_df = pd.DataFrame([summary_data])

    return metrics_df, summary_df

# Пример использования
OPENROUTER_API_KEY = userdata.get('OPEN_Router')

# Инициализация модели (меняется только здесь)
model_path = "deepseek/deepseek-chat"

# Оцениваем модель и получаем summary_df
metrics_df, summary_df = evaluate_model(df, OPENROUTER_API_KEY, model=model_path)

# Результаты
summary_df


,Model,Average BLEU,Average METEOR,Average ROUGE-1,Average ROUGE-2,Average ROUGE-L,Correct Responses (%)
0,deepseek/deepseek-chat,0.357821,0.649536,0.785041,0.645557,0.754927,0.0


In [ ]:
import pandas as pd
import requests
import json
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer

# Функция для вызова модели через API
def generate_sql_code(instruction, table_schema, api_key, model):
    prompt = f"""
    You are an expert SQL query generator. Answer only with valid SQL code, nothing else.
    Instruction: {instruction}
    Table Schema: {table_schema}
    SQL code:
    """
    input_data = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps(input_data)
    )
    if response.status_code == 200:
        data = response.json()
        return data.get('choices', [{}])[0].get('message', {}).get('content', 'No content found')
    else:
        raise RuntimeError(f"API request failed with status {response.status_code}: {response.text}")

# Функция для вычисления метрик
def calculate_metrics(true_answer, model_response):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(true_answer, model_response)

    true_tokens = true_answer.split()
    response_tokens = model_response.split()
    smooth_fn = SmoothingFunction().method1
    bleu_score = sentence_bleu([true_tokens], response_tokens, smoothing_function=smooth_fn)

    meteor = meteor_score([true_answer.split()], model_response.split())

    return bleu_score, meteor, rouge_scores['rouge1'].fmeasure, rouge_scores['rouge2'].fmeasure, rouge_scores['rougeL'].fmeasure

import sqlite3

def test_sql_query(table_schema, sql_query):
    try:
        # Создаем временную базу данных SQLite в памяти
        conn = sqlite3.connect(":memory:")
        cursor = conn.cursor()

        # Выполняем создание таблицы из схемы
        cursor.executescript(table_schema)

        # Выполняем сгенерированный SQL-запрос
        cursor.execute(sql_query)
        result = cursor.fetchall()

        # Закрываем соединение
        conn.close()

        return {"success": True, "result": result}
    except Exception as e:
        return {"success": False, "error": str(e)}

# Основной код
def evaluate_model(df, api_key, model_name="SQL Query Generator", summary_df=None):
    metrics_data = []  # Список для хранения строк данных
    for i, row in df.head(30).iterrows():  # Обрабатываем первые 30 строк
        instruction = row['instruction']
        table_schema = row['input']  # Используем описание таблицы
        true_response = row['response']

        try:
            # Генерация ответа модели
            model_response = generate_sql_code(instruction, table_schema, api_key, model_name)

            # Тестируем SQL-код
            test_result = test_sql_query(table_schema, model_response)
            sql_valid = test_result["success"]
            sql_error = test_result.get("error", "")

            # Вычисляем метрики
            bleu, meteor, rouge1, rouge2, rougeL = calculate_metrics(true_response, model_response)

            # Добавляем строку данных в список
            metrics_data.append({
                "Instruction": instruction,
                "Response": true_response,
                "Model Response": model_response,
                "BLEU": bleu,
                "METEOR": meteor,
                "ROUGE-1": rouge1,
                "ROUGE-2": rouge2,
                "ROUGE-L": rougeL,
                "SQL Valid": sql_valid,
                "SQL Error": sql_error
            })
        except Exception as e:
            print(f"Error processing row {i}: {e}")

    # Создаем датафрейм с метриками для каждого примера
    metrics_df = pd.DataFrame(metrics_data)

    # Рассчитываем средние значения метрик и процент правильных ответов
    average_bleu = metrics_df['BLEU'].mean()
    average_meteor = metrics_df['METEOR'].mean()
    average_rouge1 = metrics_df['ROUGE-1'].mean()
    average_rouge2 = metrics_df['ROUGE-2'].mean()
    average_rougeL = metrics_df['ROUGE-L'].mean()
    correct_responses_percentage = (metrics_df['SQL Valid'].sum() / len(metrics_df)) * 100

    # Создаем строку с метриками для текущей модели
    summary_data = {
        "Model": model_name,
        "Average BLEU": average_bleu,
        "Average METEOR": average_meteor,
        "Average ROUGE-1": average_rouge1,
        "Average ROUGE-2": average_rouge2,
        "Average ROUGE-L": average_rougeL,
        "Correct Responses (%)": correct_responses_percentage
    }

    # Если передан существующий summary_df, добавляем строку
    if summary_df is not None:
        summary_df = pd.concat([summary_df, pd.DataFrame([summary_data])], ignore_index=True)
    else:
        # Создаем новый датафрейм, если его нет
        summary_df = pd.DataFrame([summary_data])

    return metrics_df, summary_df

# Пример использования
OPENROUTER_API_KEY = userdata.get('OPEN_Router')

# Список моделей для оценки
models = [
    "meta-llama/llama-3.2-1b-instruct",
    "deepseek/deepseek-chat",
    "qwen/qvq-72b-preview",
    "google/gemini-2.0-flash-thinking-exp:free",
    "sao10k/l3.3-euryale-70b"
]

# Инициализация пустого summary_df
summary_df = pd.DataFrame()

# Оценка всех моделей и добавление результатов в summary_df
for model in models:
    metrics_df, summary_df = evaluate_model(df, OPENROUTER_API_KEY, model_name=model, summary_df=summary_df)

# Результаты
print(summary_df)


Error processing row 16: Response ended prematurely
                                       Model  Average BLEU  Average METEOR  \
0           meta-llama/llama-3.2-1b-instruct      0.191077        0.460096   
1                     deepseek/deepseek-chat      0.391790        0.667210   
2                       qwen/qvq-72b-preview      0.005871        0.080776   
3  google/gemini-2.0-flash-thinking-exp:free      0.009636        0.064277   
4                    sao10k/l3.3-euryale-70b      0.291350        0.612193   

   Average ROUGE-1  Average ROUGE-2  Average ROUGE-L  Correct Responses (%)  
0         0.714977         0.542291         0.659192              46.666667  
1         0.801627         0.667992         0.761502               0.000000  
2         0.040793         0.028719         0.038245               0.000000  
3         0.043895         0.032452         0.042588               0.000000  
4         0.833023         0.725874         0.806442               3.333333  


In [ ]:
summary_df

,Model,Average BLEU,Average METEOR,Average ROUGE-1,Average ROUGE-2,Average ROUGE-L,Correct Responses (%)
0,meta-llama/llama-3.2-1b-instruct,0.191077,0.460096,0.714977,0.542291,0.659192,46.666667
1,deepseek/deepseek-chat,0.391790,0.667210,0.801627,0.667992,0.761502,0.000000
2,qwen/qvq-72b-preview,0.005871,0.080776,0.040793,0.028719,0.038245,0.000000
3,google/gemini-2.0-flash-thinking-exp:free,0.009636,0.064277,0.043895,0.032452,0.042588,0.000000
4,sao10k/l3.3-euryale-70b,0.291350,0.612193,0.833023,0.725874,0.806442,3.333333
